##  Analisis Status Pemain NBA Terhadap Points yang Didapatkan

<hr>Pertama adalah import library yang dibutuhkan untuk keperluan analisis<hr>

In [ ]:
import numpy as np
import csv 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sqlalchemy import create_engine
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture as GMM

import warnings
warnings.filterwarnings('ignore')

<hr>Selanjutnya melakukan pemanggilan data <b>Status Player</b> dengan nama player_stats.csv yang berbentuk .csv kedalam jupyter notebook menggunakan library dari pandas dan menampilkan data.head() agar menampilkan data atas saja sejumlah 6 <hr>

In [ ]:
df = pd.read_csv('/kaggle/input/nba-players-stats-20142015/players_stats.csv')
df.head()

In [ ]:
df.info()

<hr>Check data yang memiliki nilai ganda atau duplikasi <hr>

In [ ]:
duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF

<hr>Melihat data yang memiliki value NaN pada setiap kolom<hr>

In [ ]:
df.isnull().any()

### i. Cleaning Data

<hr>Dalam data frame induk diatas terlihat kolom <b>"Height"</b> yang akan saya gunakan untuk melihat korelasi data, kolom "Height" memiliki isi data yang <i>NaN</i> oleh sebab itu, saya akan collecting data yang tidak memiliki value NaN pada kolom "Height" dengan cara sebagai berikut :
<ol>
    <li>Membuat Data Frame Baru dengan data BMI yang tidak NaN</li>
    <li>Kenapa menggunakan kolom BMI? karena didalam  menentukan BMI memerlukan Height dan Weight</li>
    <li>Jika BMI memiliki nilai NaN, maka secara tidak langsung Height atau Weight memiliki nilai NaN </li>
</ol>
Bagaimana dengan kolom "Weight"? Hanya untuk berjaga jaga siapa tau di kemudian hari kolom "Weight" akan digunakan<hr>

In [ ]:
dfc = df[pd.notnull(df['BMI'])]

<hr>Meihat data NaN (null) pada kolom "Height" dengan tujuan untuk melihat keseluruhan data frame yang akan digunakan untuk analisa tidak memiliki nilai NaN <hr>

In [ ]:
dfnull = dfc[dfc['Height'].isnull()]
dfnull

### ii. Data Preparing
<hr>Setelah Proses <b>Cleaning</b> selesai saya akan membuat data frame baru untuk menampung data yang telah di cleaning tersebut dengan menambahkan beberapa kolom yang akan dianalisa
<ol>
    <li><b>Menit bermain pemain setiap gamenya</b></li>
    Hal ini dilakukan karena berapa menit pemain didalam setiap game untuk mengumpulkan point sebanyak value yang ada di kolom PTS
    <li><b>Total Tembakan yang dilakukan oleh pemain</b></li>
    Berapa total tembakan yang dilakukan oleh pemain agar dapat mengumpulkan point sebanyak value yang ada di kolom PTS
</ol>
<hr>

In [ ]:
MPG = df['MIN'] / df['Games Played']
TFG = df['FGM'] + df['FGA']

<hr>Dari variabel baru diatas yang telah kita kalkulasikan, langkah selanjutnya adalah memasukan variabel diatas kedalam
kolom data frame <b>dfl</b> dengan bantuan fungsi data.insert dari library pandas

<b> Berikut adalah Data Frame yang sudah siap untuk di analisa </b>
<p> Terdapat tambahan menit bermain setiap game dan total tembakan yang dilakukan
<hr>

In [ ]:
dfl = (dfc[["Name", "Team", "Games Played", "MIN", "EFF", "FGM", "FGA", "PTS"]].sort_values("PTS", ascending=False))
dfl.insert(4, 'PMPG', MPG)
dfl.insert(8, 'TFG', TFG)

dfl.head()

<br>
<b><center>Penjelasan Nama Kolom Diatas</center></b>
<table>
  <tr>
    <td><b>Name</b></td>
    <td>Nama Pemain</td>
    <td><b>Team</b></td>
    <td>Team Pemain</td>
    <td><b>Game Played</b></td>
    <td>Banyaknya Game yang Dimainkan</td>
    <td><b>MIN</b></td>
    <td>Menit Bermain</td>
    <td><b>PMPG</b></td>
    <td>Jumlah Menit Bermain Setiap Game</td>
  </tr>
  <tr>
    <td><b>EFF</b></td>
    <td>Effisiensi Pemain</td>
    <td><b>FGM</b></td>
    <td>Total Tembakan Masuk</td>
    <td><b>FGA</b></td>
    <td>Total Tembakan Bebas</td>
    <td><b>TFG</b></td>
    <td>Total Keseluruhan Tembakan</td>
    <td><b>PTS</b></td>
    <td>Point yang Disumbangkan Pemain</td>
  </tr>
</table>

In [ ]:
# dfl.isnull().any()

## 1. Buat Scatter Plot untuk Masing Masing Feature 
<hr>
Setelah data frame yang akan digunakan untuk analisis sempurna maka langkah selanjutnya adalah melihat korelasi beberapa data terhadap kolom point "PTS", untuk memudahkan dalam visualisasi dalam kasus ini saya menggunakan library dari <b>seaborns</b> yang terintegrasi dengan matplotlib dan pandas

<p>Karena banyaknya feature yang ada, visualisasi scatter plot menggunakan seaborns saya lakukan agar pengerjaan scatter dapat berjalan secara efektif dan efisien dalam menganalisa</p><hr>

In [ ]:
sns.pairplot(dfl)

## 2. Dugaan Feature yang Memiliki Korelasi Kuat
<hr>
<p> Dengan melihat scatter plot pada gambar diatas terdapat 2 visualisasi yang memiliki korelasi kuat yaitu :
    <ol>
        <li>Menit bermain pemain (MIN) terhadap point yang disumbangkan bagi TIM</li>
        <li>Effisiensi bermain pemain (EFF) dalam menyumbangkan point untuk TIM </li>
    </ol>
</p>
<hr>

<b> MIN - Menit Bermain </b>
<p>Menit bermain adalah kalkulasi dari pemain yang masuk dari quarter ke-1 hingga ke-4 dan setiap quarternya memiliki waktu 12 menit, pada data yang disajikan terdapat catatan menit bermain pemain selama satu musim yang dapat dikorelasikan dengan point. Pada situs <a href="https://www.basketball-reference.com/leagues/NBA_2015.html">Basket Ball Reference - Tim Stats 2014/2015</a> dimana menit bermain pada tim tidak selalu mempengaruhi point yang didapatkan</p>

<b>EFF - Effisiensi</b>
<p>EFF atau yang disebut Effisiensi Pemain dalam bermain bola basket memiliki relasi dengan point yang didapatkan dikarenakan menurut analisa sederhana saya pada situs <a href="http://www.espn.com/nba/hollinger/teamstats">ESPN Team Statistic</a> tim dengan Effisiensi menyerang tertinggi menduduki peringkat pertama klasemen <i>National Basketball Association</i> pada musim 2018-2019 dan dikuatkan dengan pengertian yang ada pada <a href="https://en.wikipedia.org/wiki/Efficiency_(basketball)#EFF">Wikipedia</a>dimana penghitungan effisiensi dapat dilakukan pada point yang di cetak</p>

<hr>

<b>Kesimpulan :</b> 
<p>Saya putuskan untuk menggunakan data Effisiensi dikarenakan pada rumus kalkulasi effisiensi terdapat beberapa variabel status pemain yang relevan digunakan seperti jumlah rebound, assist, steal, block, free throw, dan turnover. Setiap status yang saya sebutkan tersebut memiliki hubungan erat dengan point yang didapatkan.</p>

<hr>

In [ ]:
dfl.plot.scatter("EFF", "PTS", alpha=0.5, color= "r", figsize=(13,5))
plt.xlabel('Effisiensi Bermain')
plt.ylabel('Point')

## 3. Statistik Deskriptif EFF Terhadap Point
Pada statistik deskriptif ini saya membagi efisiensi menjadi 3 bagian, dikarenakan pada olah raga bola basket tim terbagi menjadi beberapa regional dan pada setiap team terdapat God Among Man (MVP Player)
<ol>
    <li>Effisiensi Pemain Tertinggi didalam Team pada Regional </li>
    <li>Pengaruh Effisiensi Team Terhadap Point yang Didapatkan</li>
    <li>Pengaruh Effisiensi Seluruh Pemain untuk Mendapatkan Peringkat 1 pada Regionalnya</li>
</ol>
<hr>

### A. Effisiensi Tertinggi Pemain dalam Team Regional Timur Terhadap Point yang Didapatkan
Melakukan indexing data dimana terdapat beberapa team yang tergabung di Regional Team Seperti Boston Celtics, Detroit Pistons, Cleveland Cavalirs, Orlando Magic, Toronto Raptors, Washington Wizards, Philadelphia 76ers, Milwaukee Bucks, New York Knicks, Atlanta Hawks, Charlote Hornets, Chicago Bulls, Indiana Pacers, Miami Heat. Indexing ini dimasukkan kedalam Data Frame baru yang bernama dfeast.

Pembagian tim ini dapat di akses melalui link <a href="https://id.wikipedia.org/wiki/Wilayah_Timur_(NBA)">Wikipedia NBA</a>


<hr>

In [ ]:
Boston = dfl.Team == 'BOS'
Detroit = dfl.Team == 'DET'
Cleveland = dfl.Team == 'CLE'
Orlando = dfl.Team == 'ORL'
Toronto = dfl.Team == 'TOR'
Washington = dfl.Team == 'WAS'
Philadelphia = dfl.Team == 'PHI'
Milwaukee = dfl.Team == 'MIL'
NewYork = dfl.Team == 'NYK'
Atlanta = dfl.Team == 'ATL'
Charlote = dfl.Team == 'CHA'
Chicago = dfl.Team == 'CHI'
Indiana = dfl.Team == 'IND'
Miami = dfl.Team == 'MIA'

dfeast = dfl[Boston | Detroit | Cleveland | Orlando | Toronto | Washington | Philadelphia | 
            Milwaukee | NewYork | Atlanta | Charlote | Chicago | Indiana | Miami]

# dfeast.sort_values("Team", ascending=True)

<hr>Mencari poin tertinggi dan kaitanya dengan effisiensi pemain<hr> 

In [ ]:
maxt = dfeast.groupby(['Team'])['PTS'].transform(max) == dfeast['PTS']
dfmax = dfeast[maxt]
dfmax.sort_values("EFF", ascending=False)

<hr> Membuat grafik berbentuk stacked bar untuk membandingkan kedua variabel antara effisiensi dan points <hr>

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go

tr1 = go.Bar(
                 x = dfmax['Name'],
                 y = dfmax['EFF'],
                 name = 'Effisiensi',
                 marker = dict(color='crimson',
                              line = dict(color='rgba(0,0,0)', width=0.5)),
                 text = dfmax.Team)

tr2 = go.Bar(
                 x = dfmax['Name'],
                 y = dfmax['PTS'],
                 name = 'Points',
                 marker = dict(color='rgba(0, 0, 255, 0.5)',
                              line = dict(color='rgba(0,0,0)', width=0.5)),
                 text = dfmax.Team)
dn = [tr1, tr2]
layoutnew = go.Layout(barmode='group', title='Effisiensi Tertinggi Pemain pada TIM Regional Timur Terhadap Points')
fig = go.Figure(data=dn, layout=layoutnew)
fig.update_layout(barmode='stack')
py.iplot(fig)

<hr><b>Analisa :</b>
Point Tertinggi yang dicetak oleh pemain yang menjadi MVP didalam team regional timur berbanding lurus dengan point yang didapatkan, dimana effisiensi yang
diusahakan setiap pemain akan mendapatkan point tertinggi <b>kecuali</b> pada pemain Isaiah Thomas : 950/1101, Carmelo Anthony 824/966, dan CJ Miles 715/942 (EFF/PTS) <hr>

### B. Jumlah Efffisiensi Team terhadap Jumlah Point yang Didapatkan
<hr>
Jika point diatas adalah effisiensi tertinggi pemain pada tim regional timur, maka saya akan memperlebar range atau jarak berdasarkan seluruh pemain yang ada didalam setiap Team untuk dianalisa perbandingan tim yang memiliki point yang banyak terhadap jumlah effisiensi yang dilakukan
<hr>

Pertama menjumlahkan seluruh effisiensi pemain yang tergabung berdasarkan grup Team pada regional timur menggunakan SQL <hr>

In [ ]:
engine= create_engine('sqlite:///:memory:')

dfeast.to_sql('data_table', engine) 
dfeastrt= pd.read_sql_query('SELECT SUM("EFF"), Team FROM data_table group by Team', engine)
K = dfeastrt['SUM("EFF")']

<hr>Kedua menjumlahkan seluruh point pemain yang tergabung berdasarkan grup dari Team regional timur menggunakan SQL<hr>

In [ ]:
engine= create_engine('sqlite:///:memory:')

dfeast.to_sql('data_table', engine) 
dfeastrp= pd.read_sql_query('SELECT SUM("PTS"), Team FROM data_table group by Team', engine)

<hr>Ketiga memasukkan variabel baru bernama SEFF (Total Effisiensi) dan mengganti nama SUM PTS menjadi SPTS (jumlah keseluruhan point yang didapatkan)<hr>

In [ ]:
dfeastrp.insert(1, 'SEFF', K)
dfeastrp.rename(
    columns={
        'SUM("PTS")': "SPTS"
    },
    inplace=True)

<hr>Keempat membuat kolom baru bernama Rank yang diambil dari jumlah point terbesar sampai terkecil. dan hasil dari tersebut akan menjadi Data Frame baru bernama dfeastrp yang akan digunakan untuk visualisasi dan analisis<hr>

In [ ]:
dfeastrp['Rank'] = dfeastrp['SPTS'].rank(method='dense', ascending=False)
dfeastrp.sort_values("SPTS", ascending=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfeastrp['Team'], y=dfeastrp['SEFF'], fill='tozeroy',name = 'Jumlah Effisiensi Tim'))
fig.add_trace(go.Scatter(x=dfeastrp['Team'], y=dfeastrp['SPTS'], fill='tonexty',name = 'Jumlah Point Tim'))

fig.show()

<hr>
<b>Analisa :</b> pada fill chart / area chart semakin tinggi effisiensi pemain yang dilakukan pada setiap team akan berdampak pada point yang didapatkan dan akan berbanding lurus dengan rangking team pada regional tersebut. Dapat dilihat Team Atalanta memiliki efisiensi tertinggi dalam bermain bola basket dan menduduki peringkat pertama dalam grup regional timur. dan pada klub Miami memiliki effisiensi paling sedikit diantara team yang lain dan menduduki rangking terendah
<hr>

### C. Rata Rata Point per Game yang didapatkan Team Sesuai Ranking
<hr>
Setelah saya mengetahui hubungan antara effisiensi terhadap point yang didapatkan maka relasi effisiensi dan point begitu kuat untuk mendapatkan rank didalam suatu regional, lalu kita bisa lanjut untuk menganalisa berapa presentase point pergame agar team menjadi ranking 1 di regionalnya agar nantinya dapat dilihat juga berapa effisiensi yang harus dilakukan setiap team per gamenya, dan dapat ditarik jauh lagi seperti berapa banyak effisiensi pemain yang harus dilakukan agar team mendapat ranking tertinggi
<hr>

Pertama kita akan memasukan kolom game yang dimainkan setiap team kedalam Data Frame dfeastrp menggunakan SQL<hr>

In [ ]:
engine= create_engine('sqlite:///:memory:')

dfeast.to_sql('data_table', engine) 
gp= pd.read_sql_query('SELECT "Games Played" FROM data_table group by Team', engine)

<hr>Kedua memasukan kolom Games Played kedalam dfeastrp untuk tujuan visualisasi<hr>

In [ ]:
dfeastrp.insert(3, 'GP', gp)

<hr>
Ketiga melihat Games Played tertinggi yang dilakukan, setelah itu membagi point yang di peroleh per game (Game yang dijalankan adalah 83). Membuat Data Frame baru untuk tujuan visualisasi
<hr>

In [ ]:
dfeastgpp = dfeastrp['SPTS'] / 82
dfeastgpp
# dfeastrp['GP'].max()

In [ ]:
fig = {
        'data': [ 
             {
                'values' : dfeastgpp,
                'labels' : dfeastrp['Rank'],
                'domain' : {'x': [0, 1]},
                'name' : 'Points / Game',
                'hoverinfo' : 'label+percent+name',
                'hole' : 0.3,
                'type' : 'pie'
              },
             ],
         'layout' : {
                     'title' : 'Rata Rata Point per Game yang didapatkan Team Sesuai Ranking',
                     'annotations' : [
                                        { 'font' : {'size' : 20},
                                          'showarrow' : False,
                                          'text' : ' ',
                                          'x' : 0.20,
                                          'y' : 1
                                         },
                                      ]    
                     }
        }
py.iplot(fig)

<hr>
<b>Analisa :</b> Maka dapat disimpulkan bahwa point yang dikumpulkan setiap gamenya harus berjarak 0.5% dari peringkat atas agar mampu membawa team menjadi ranking 1 pada regionalnya
<hr>

In [ ]:
dfatl = dfeast[dfeast['Team'] == "ATL"]
dfatl

<hr>
<b>Kesimpulan Akhir :</b> Semakin tinggi Effisiensi pemain didalam tim maka  dapat berkontribusi kepada team untuk berada di ranking teratas pada regionalnya. sebagai contoh adalah team Atlanta yang menduduki peringkat satu di regional timur karena memiliki point terbanyak
<hr>

## 2. Model Regresi EFF Terhadap Point
<hr>
Data Frame dflterdiri dari 2 variabel. Kolom pertama sebagai Efisiensi dan kolom kedua sebagai point yang nilainya akan kita prediksi berdasarkan inputan effisiensi independen.

Pertama kita memulai dengan membuat scatter pada data frame tersebut <hr>

In [ ]:
dfl.plot.scatter("EFF", "PTS", alpha=0.5, color= "r", figsize=(13,5))
plt.xlabel('Efisiensi Pemain')
plt.ylabel('Point')

### A.  Linier Rergression
<hr>
Sepertinya scatter yang terlihat bisa di regresikan dengan mode Linier.

Pertama Membuat data train yang diambil dari data utama sebanyak 20% dari total keseluruhan data. untuk kepereluan regresi linier dan ditampilkan dalam bentuk scatter.
<hr>

In [ ]:
msk = np.random.rand(len(dfl)) < 0.8
train = dfl[msk]
test = dfl[~msk]

plt.figure(figsize=(13,5))
plt.scatter(train.EFF, train.PTS, alpha=0.5, color='blue')
plt.xlabel("Efisiensi Pemain")
plt.ylabel("Point")
plt.show()

<hr>Kedua adalah menampilkan nilai Coefficients dan Intercept menggunakan fungsi yang sudah disediakan (default).

Maka akan mendapatkan vektor prediksi sebagai berikut <hr>

In [ ]:
regr = linear_model.LinearRegression()
train_x = np.asanyarray(train[['EFF']])
train_y = np.asanyarray(train[['PTS']])
regr.fit (train_x, train_y)

print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

<hr>Ketiga setelah kita mendapatkan B0 dan B1 nya maka kita dapat memvisualisasikan vektor prediksi tersebut kedalam scatter<hr>

In [ ]:
plt.figure(figsize=(13,5))
plt.scatter(train.EFF, train.PTS, alpha=0.5, color='blue')
plt.plot(train_x, regr.coef_[0][0]*train_x + regr.intercept_[0], '-r')
plt.xlabel("Efisiensi Pemain")
plt.ylabel("Point")

<hr>
Keempat kita uji coba B0 dan B1 tersebut kedalam rumus <b>𝑦 ̂=𝜃_0+𝜃_1  𝑥_1</b> untuk melihat kualitas regresi yang dihasilkan
<hr>

In [ ]:
testk = regr.intercept_ + regr.coef_ * 1475
testk

<hr><b>Kesimpulan :</b> Tenyata dari Regresion diatas memiliki tingkat error yang banyak oleh sebab itu Linier Regression Model tidak cocok digunakan untuk pola dari scatter ini <hr>

In [ ]:
from sklearn.metrics import r2_score

test_x = np.asanyarray(test[['EFF']])
test_y = np.asanyarray(test[['PTS']])
test_y_ = regr.predict(test_x)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y_ , test_y) )

### B.  Non-Linier Rergression
<hr>
Jika Linier Regression memiliki error yang tinggi maka dapat disimpulkan ada lengkungan (tidak lurus) saat penyajian data dalam bentuk scatter oleh sebab itu saya mencoba untuk menggunakan Regresi Non Linier
<hr>

In [ ]:
plt.figure(figsize=(13,5))
x_data, y_data = (dfl["EFF"].values, dfl["PTS"].values)
plt.plot(x_data, y_data, 'mo' ,alpha=0.5)
plt.xlabel('Efisiensi Pemain')
plt.ylabel('Point')
plt.show()

<hr>Pertama adalah membuat class non linier regression terdiri dari
<ol>
    <li>Sigmoid</li>
    <li>Exponensial</li>
    <li>Qubic</li>
</ol>
    Dengan menggunakan rumus
    <center>
        <b>Sigmoid</b> <br> $$ Y = a + \frac{b}{1+ c^{(X-d)}}$$ <br><br>
        <b>Exponensial</b> <br> $$ \hat{Y} = \frac1{1+e^{\beta_1(X-\beta_2)}}$$ <br><br>
        <b>Qubic</b> <br> $$ \ y = a x^3 + b x^2 + c x + d \ $$
    </center>
<hr>

In [ ]:
def sigmoid(x, Beta_1, Beta_2):
     y = 1 / (1 + np.exp (-beta_1*(x-Beta_2)) )
     return y

def expo (x, Beta_0, Beta_1):
     y = Beta_0*np.exp(Beta_1*x)
     return y

def qubic (x, Beta_0, Beta_1, Beta_2, Beta_3):
     y = Beta_0+Beta_1*x+Beta_2*x**2+Beta_3*x**3
     return y

In [ ]:
beta_1 = 1.0
beta_2 = 1
beta_3= 1
beta_4=0.1

Y_preds =sigmoid (x_data, beta_1, beta_2)


<hr>
Kedua membuat variabel x dan y dimana x = datax : maksimal(datax) dan x = datax : maksimal(datax)
<hr>

In [ ]:
xdata =x_data/max(x_data)
ydata =y_data/max(y_data)

<hr>Ketiga yaitu setelah semua rumus dimasukkna kedalam class dan didefinisikan maka langkah selanjutnya adalah Mencari B0 (Inter) dan B1 (Coef) menggunakan rumus 
<hr>

In [ ]:
from scipy.optimize import curve_fit
popt1, pcov1 = curve_fit(sigmoid, xdata, ydata, maxfev = 10000)
popt2, pcov2 = curve_fit (expo, xdata, ydata, maxfev = 10000) 
popt3, pcov3 = curve_fit (qubic, xdata, ydata, maxfev = 10000) 

print(" Exponensial","B1 = %f, B2=%f"%(popt2[0], popt2[1]))
print(" Sigmoid","B1 = %f, B2=%f"%(popt1[0], popt1[1]))
print(" Qubic","B1 = %f, B2=%f"%(popt3[0], popt3[1]))

<hr>
Keempat setelah nilai B0 dan B1 maka kita buat scatternya sebagai berikut terhadap line dari (Eksponensial, Sigmoid, dan Qubic) dan mencari Error terkecil
<hr>

In [ ]:
x = np.linspace(10, 200, 1000)
x = x/max(x)
plt.figure(figsize=(13,5))

y1 = expo(x, *popt1)
y2 = sigmoid(x, *popt2)
y3 = qubic(x, *popt3)

plt.plot(xdata, ydata, 'mo',alpha=0.5 ,label='data')
plt.plot(x,y1, linewidth=3.0, label='Eksponensial')
plt.plot(x,y2, linewidth=3.0, label='Sigmoid')
plt.plot(x,y3, linewidth=3.0, label='Qubic')
plt.legend(loc='best')
plt.xlabel('Efisiensi Pemain')
plt.ylabel('Points')
plt.show()

In [ ]:
msk = np.random.rand(len(dfl)) < 0.8

train_x = xdata[msk]
test_x = xdata[~msk]
train_y = ydata[msk]
test_y = ydata[~msk]

# build the model using train set
popt1, pcov1 = curve_fit(sigmoid, train_x, train_y, maxfev = 100000)
popt2, pcov2 = curve_fit(expo, train_x, train_y, maxfev = 100000)
popt3, pcov3 = curve_fit(qubic, xdata, ydata, maxfev = 10000)

y_hat1 = sigmoid(test_x, *popt1)
y_hat2 = expo(test_x, *popt2)
y_hat3 = qubic(test_x, *popt3)

print ("Sigmoid")
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat1 - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat1 - test_y) ** 2))
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_hat1 , test_y) )

print ("\nExponens")
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat2 - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat2 - test_y) ** 2))
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_hat2 , test_y) )

print ("\nQubic")
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat3 - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat3 - test_y) ** 2))
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_hat3 , test_y) )

<b>Kesimpulan :</b> Error terkecil berada pada Regresi Non Linier milik Qubic hanya 0,5 dibandingkan dengan yang lainnya. Prediksi dapat dilakukan menggunakan hasil B0 dan B1 dari regresi Qubic

In [ ]:
x = np.linspace(10, 200, 1000)
x = x/max(x)
plt.figure(figsize=(13,5))

y3 = qubic(x, *popt3)

plt.plot(xdata, ydata, 'mo',alpha=0.5 ,label='data')
plt.plot(x,y3, linewidth=3.0, label='Qubic')
plt.legend(loc='best')
plt.xlabel('Menit Bermain')
plt.ylabel('Points')
plt.show()

In [ ]:
print ("\nQubic")
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat3 - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat3 - test_y) ** 2))
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_hat3 , test_y) )

## 5 Analisis Menggunakan Clustering
<hr>
Analisis clustering ini bertujuan untuk mengklasifikasi posisi pemain pada musim berikutnya sesuai dengan status individu yang relevan untuk posisinya sebagai pemain basket

Dimana klasifikasi juga dilakukan dengan rumus penentuan x = asist/menit bermain dan y = rebound/menit sesuai pada rumus yang ada di website <a href="https://medium.com/@vmusgrove86/a-data-dive-into-2018-2019-nba-player-stats-in-python-d7a1c051ac5c">NBA Medium</a>, penjelasan tentang posisi bola pemain bola basket dapat dilihat pada situs <a href="https://id.wikipedia.org/wiki/Posisi_bola_basket">Wikipedia</a>

<ol>
    <li>SG : Shooting Guard</li>
    <li>PF : Power Forward</li>
    <li>PG : Point Guard</li>
    <li>C  : Center</li>
    <li>SF  : Small Forward</li>
</ol>
<hr>

Pertama yang dilakukan adalah menghapus beberapa pemain yang memiliki nilai X dan Y tidak relevan dengan clustering nantinya
<hr>

In [ ]:
dfdel1 = dfc.Name != 'Sim Bhullar'
dfdel2 = dfc.Name != 'Jerrelle Benimon'
dfcl = dfc[dfdel1 & dfdel2]

<hr>
Kedua Membuat Rumus Total Rebound (Rebound/Menit) dan Total Asist (Asist/Menit), lalu mendeklarasikan 2 point tersebut kedalam variabel baru bernama x_var dan y_var.

Selanjutnya adalah membuat grafik scatter untuk melihat pola yang akan diclustering nantinya (pewarnaan bisa dilihat melalui code dibawah) 
<hr>

In [ ]:
dfcl["TRB/MIN"] = dfcl["REB"]/dfcl["MIN"] 
dfcl["AST/MIN"] = dfcl["AST"]/dfcl["MIN"]

fig, ax = plt.subplots()

x_var="AST/MIN"
y_var="TRB/MIN"

colors = {'SG':'blue', 'PF':'red', 'PG':'green', 'C':'purple', 'SF':'orange'}

ax.scatter(dfcl[x_var], dfcl[y_var], c=dfcl['Pos'].apply(lambda x: colors[x]), s = 10)

# set a title and labels
ax.set_title('NBA Dataset')
ax.set_xlabel(x_var)
ax.set_ylabel(y_var)

<hr>Ketiga ketika scatter color telah terbentuk maka langkah selanjutnya adalah clustering menggunakan library dari KMeans dengan asumsi 5 cluster sesuai warna yang muncul (Hijau/PG, Biru/SG, Orange/SF, Ungu/C, Merah/PF).

Maka akan muncul 5 pembagian pada x dan y<hr>

In [ ]:
dfn = dfcl[["AST/MIN","TRB/MIN"]]

kmeans = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 500, n_init = 10, random_state = 0)
y_kmeans = kmeans.fit_predict(dfn)
print(kmeans.cluster_centers_)

<hr>Keempat setelah clustering dijalankan maka akan saya tampilkan menggunakan scatter sesuai dengan cluster yang sudah terbentuk [d0, d1, d2, d3, d4] maka hasilnya akan terlihat seperti dibawah<hr>

In [ ]:
d0=dfn[y_kmeans == 0]
d1=dfn[y_kmeans == 1]
d2=dfn[y_kmeans == 2]
d3=dfn[y_kmeans == 3]
d4=dfn[y_kmeans == 4]

plt.scatter(d0[x_var], d0[y_var], s = 10, c = 'blue', label = 'D0')
plt.scatter(d1[x_var], d1[y_var], s = 10, c = 'green', label = 'D1')
plt.scatter(d2[x_var], d2[y_var], s = 10, c = 'red', label = 'D2')
plt.scatter(d3[x_var], d3[y_var], s = 10, c = 'purple', label = 'D3')
plt.scatter(d4[x_var], d4[y_var], s = 10, c = 'orange', label = 'D4')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], s = 100, c = 'yellow', label = 'Centroids')

<hr>Kelima setelah clustering pada grafik sudah terlihat jelas maka variabel x yang berupa angka akan saya ubah menjadi nama posisi didalam bola basket untuk memudahkan dalam membaca data nantinya, dan menyatuka type tuples tersebut kedalam suatu variabel<hr>

In [ ]:
d0[x_var]='SG'
d1[x_var]='PF'
d2[x_var]='PG'
d3[x_var]='C'
d4[x_var]='SF'

dflist = pd.concat([d0[x_var], d1[x_var], d2[x_var], d3[x_var], d4[x_var]])

<hr>
Terakhir yaitu membuat Data Frame untuk melihat keselurahn data akhir, dimana saya memasukan Nama, Team, Posisi, dan menambahkan  total rebound dan asist.

Data dari cluster yang telah dibuat dimasukan setelah Posisi dengan nama kolom Next Pos hal ini bertujuan untuk memaksimalkan potensi pemain NBA pada musim 2015/2016 dengan kaitannya status pemain pada musim 2014/2015
<hr>

In [ ]:
dfcluster = (dfc[["Name", "Team", "Pos"]])
dfcluster

dfcl["TRB/MIN"]
dfcl["AST/MIN"]

dfcluster.insert(2, 'TRBMIN', dfcl["TRB/MIN"])
dfcluster.insert(3, 'ASTMIN', dfcl["AST/MIN"])
dfcluster.insert(5, 'Next Pos', dflist)
dfcluster